# 3D classification based on DenseNet

Based on a tutorial by the MONAI Consortium: https://github.com/Project-MONAI/tutorials/blob/main/3d_classification/densenet_training_array.ipynb

## Setup imports

In [ ]:
import logging
import os
import glob
import sys
import shutil
import tempfile
import datetime
import socket
import functools

from tqdm import tqdm

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import nibabel as nib
import pydicom

import torch
from torch.utils.tensorboard import SummaryWriter

from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, roc_auc_score

import monai
from monai.apps import download_and_extract
from monai.config import print_config
from monai.data import DataLoader, ImageDataset
from monai.transforms import (
    EnsureChannelFirst,
    Compose,
    RandRotate90,
    Resize,
    ScaleIntensity,
)

pin_memory = torch.cuda.is_available()
torch.backends.cudnn.benchmark = False #torch.cuda.is_available() # Set this to true if the code fails
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
print_config()
device

## Helper functions to get predictions/confusion matrices

In [ ]:
def get_predicted_values(model, loader):
    # Iterates through a dataloader and gets the prediction and labels
    t_model_outputs = []
    t_test_labels = []
    for test_data in loader:
        test_images, test_labels = test_data[0].to(device), test_data[1].to(device)
        with torch.no_grad():
            val_outputs = model(test_images)
            val_outputs_argmax = val_outputs.argmax(dim=1)
            t_model_outputs.append(val_outputs_argmax.cpu().detach().numpy())
            t_test_labels.append(test_labels.cpu().detach().numpy())
    conf_model_outputs = np.concatenate(t_model_outputs)
    conf_test_labels = np.concatenate(t_test_labels)
    return conf_model_outputs, conf_test_labels

def get_cm(conf_model_outputs, conf_test_labels, num_classes):
    # Get a (count) confusion matrix based on the predictions and true labels
    conf_matrix = np.zeros((num_classes,num_classes))
    for i in range(num_classes):
        in_class_i = conf_test_labels==i
        for j in range(num_classes):
            in_class_i_predicted_in_class_j = sum(conf_model_outputs[in_class_i]==j)
            conf_matrix[i,j] = in_class_i_predicted_in_class_j
    return conf_matrix

## Set up data

The code expects the data to be inside a directory in the user's home directory. Data is from the [medical decathlon](http://medicaldecathlon.com/) and can be downloaded from https://drive.google.com/drive/folders/1HqEgzS8BV2c7xYNrZdEAnrHk7osJJ--2?usp=sharing

In [ ]:
root_dir = os.path.expanduser(os.path.expandvars("~/data/medicaldecathlon/"))
data_dir = os.path.join(root_dir, "Task10_Colon")
train_dataset_frailty_path = os.path.join(data_dir,"train_clean.csv")
test_dataset_frailty_path = os.path.join(data_dir,"test_clean.csv")
print(root_dir)

Get list of images

In [ ]:
train_images = sorted(
    glob.glob(os.path.join(data_dir, "imagesTr", "*.nii.gz")))
train_images[:5]

In [ ]:
test_image_paths = sorted(
    glob.glob(os.path.join(data_dir, "imagesTs", "*.nii.gz")))
test_image_paths[:5]

Get training and testing labels. You should move these from this repo's `data` folder to the same folder as that contain the image folders above.

In [ ]:
df_labels = pd.read_csv(train_dataset_frailty_path, index_col="PatientID").dropna()
df_labels.head()

In [ ]:
df_labels_test = pd.read_csv(test_dataset_frailty_path, index_col="PatientID").dropna()
df_labels_test.head()

Change values in the dataframes from string to class ids

In [ ]:
df_labels.loc[df_labels["Risk Category"]=="LOW","Risk Category"] = 0
df_labels.loc[df_labels["Risk Category"]=="MEDIUM","Risk Category"] = 1
df_labels.loc[df_labels["Risk Category"]=="HIGH","Risk Category"] = 2

df_labels_test.loc[df_labels_test["Risk Category"]=="LOW","Risk Category"] = 0
df_labels_test.loc[df_labels_test["Risk Category"]=="MEDIUM","Risk Category"] = 1
df_labels_test.loc[df_labels_test["Risk Category"]=="HIGH","Risk Category"] = 2

In [ ]:
def get_id_from_filepath(fpath):
    # Gets patient ID from its filepath
    return int(os.path.basename(fpath).split("_")[1].split(".")[0])

Drop any patients whose scans have less than this number of slices

In [ ]:
MIN_SLICES = 64

Get image and label information on a single list

In [ ]:
data_dicts = [
    {"image": image_name, "label": df_labels.loc[patient_id,"Risk Category"]}
    for image_name,patient_id in zip(train_images,map(get_id_from_filepath,train_images))
    if patient_id in df_labels.index and nib.load(image_name).get_fdata().shape[2]>=MIN_SLICES
]
len(data_dicts)

In [ ]:
test_data_dicts = [
    {"image": image_name, "label": df_labels_test.loc[patient_id,"Risk Category"]}
    for image_name,patient_id in zip(test_image_paths,map(get_id_from_filepath,test_image_paths))
    if patient_id in df_labels_test.index and nib.load(image_name).get_fdata().shape[2]>=MIN_SLICES
]
len(test_data_dicts)

Separate image and labels into their own lists, get the number of labels we have (number of classes)

In [ ]:
images = np.array([d["image"] for d in data_dicts])
labels = np.array([d["label"] for d in data_dicts])

test_images = [d["image"] for d in test_data_dicts]
test_labels = [d["label"] for d in test_data_dicts]

# Number of different classes we have in the dataset
num_labels = int(max(labels))+1
num_labels, labels[:5], test_labels[:5]

Check the minimum shape of the scans

In [ ]:
shapes = [nib.load(datapoint["image"]).get_fdata().shape for datapoint in data_dicts]
np_shapes = np.stack(shapes)
minshapes = np.min(np_shapes, axis=0)
minshapes

The original code cropped images to a 96x96x96 image, since a significant amount of our images don't have that size, we need to adjust the crop to fit our training images' size.

In [ ]:
original_crop = 96
for newcrop in range(original_crop,2,-1):
    if newcrop <= min(minshapes):
        crop_shapes = tuple([newcrop]*3)
        break
crop_shapes

Set up the validation with `val_pct` images being allocated to the validation dataset with a stratified split

In [ ]:
val_pct = 0.1
val_split = int(val_pct*len(labels))
possible_labels = sorted(list(set(labels)))
proportion_in_labels = np.array([sum(labels==i)/len(labels) for i in possible_labels])
val_per_labels = [int(l*val_split) for l in proportion_in_labels]
proportion_in_labels, val_per_labels

Split the dataset into validation/training

In [ ]:
val_idx = np.concatenate([np.random.choice([i for i, l in enumerate(labels) if l==p], c, replace=False) for p,c in zip(possible_labels, val_per_labels)])
in_val = np.isin(np.arange(len(labels)),val_idx)
in_train = ~in_val
train_idx = np.arange(len(labels))[in_train]
val_idx

Quick fix to make sure training doesn't crash

In [ ]:
batch_size = 4
while (len(images)-val_split)%batch_size==1 or val_split%batch_size==1:
    batch_size +=1
    print("Changing batch size so that no batch has size 1, as single-instance batches cause an error")
batch_size

Define training and validation transforms

In [ ]:
# Define transforms
train_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize(crop_shapes), RandRotate90()])

val_transforms = Compose([ScaleIntensity(), EnsureChannelFirst(), Resize(crop_shapes)])

# Define nifti dataset, data loader
check_ds = ImageDataset(image_files=images, labels=labels, transform=train_transforms)
check_loader = DataLoader(check_ds, batch_size=3, num_workers=2, pin_memory=pin_memory)

im, label = monai.utils.misc.first(check_loader)
print(type(im), im.shape, label, label.shape)

# create a training data loader
train_ds = ImageDataset(image_files=images[train_idx].tolist(), labels=labels[train_idx].tolist(), transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=pin_memory)

# create a validation data loader
val_ds = ImageDataset(image_files=images[val_idx].tolist(), labels=labels[val_idx].tolist(), transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=batch_size, num_workers=2, pin_memory=pin_memory)

In [ ]:
len(images)

In [ ]:
len(train_ds), len(val_ds)

In [ ]:
len(train_loader), len(val_loader)

Define metrics to follow-up the training as well as validation metric to save the best model

In [ ]:
metrics_fns = {
    (s.func.__name__ if isinstance(s, functools.partial) else s.__name__).split("_score")[0]: s
    for s in [functools.partial(f1_score, average="weighted"), accuracy_score, balanced_accuracy_score]
}
val_metric = "accuracy"
metrics_fns

In [ ]:
# Create DenseNet121, CrossEntropyLoss and Adam optimizer
model = monai.networks.nets.DenseNet(spatial_dims=3, in_channels=1, out_channels=num_labels).to(device)

In [ ]:
loss_function = torch.nn.CrossEntropyLoss()#torch.tensor(1/proportion_in_labels, device=device, dtype=torch.float32)
# loss_function = torch.nn.BCEWithLogitsLoss()  # also works with this data

optimizer = torch.optim.Adam(model.parameters(), 1e-4)

# start a typical PyTorch training
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
writer = SummaryWriter(f"runs{num_labels}/{datetime.datetime.now():%Y-%m-%d_%H:%M:%S}_{socket.gethostname()}")
max_epochs = 256

for epoch in range(max_epochs):
    try:
        print("-" * 10)
        print(f"epoch {epoch + 1}/{max_epochs}")
        model.train()
        epoch_loss = 0
        step = 0

        for batch_data in train_loader:
            step += 1
            b_inputs, b_labels = batch_data[0].to(device), batch_data[1].to(device)
            optimizer.zero_grad()
            outputs = model(b_inputs)
            loss = loss_function(outputs, b_labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            epoch_len = len(train_ds) // train_loader.batch_size
            print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
            writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)

        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

        if (epoch + 1) % val_interval == 0:
            model.eval()

            num_correct = 0.0
            metric_count = 0
            v_pred, v_label = get_predicted_values(model, val_loader)
            cm_counts = get_cm(v_pred, v_label, num_labels)

            all_metrics = {
                metric: metrics_fns[metric](v_label, v_pred) for metric in metrics_fns
            }
            metric = all_metrics[val_metric]
            metric_values.append(metric)

            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), f"best_metric_model{num_labels}_classification3d_array.pth")
                print("saved new best metric model")

            print(f"Current epoch: {epoch+1} current {val_metric}: {metric:.4f} ")
            print(f"Best {val_metric}: {best_metric:.4f} at epoch {best_metric_epoch}")
            for metric in all_metrics:
                writer.add_scalar(f"val_{metric}", all_metrics[metric], epoch + 1)
            cm_pct = cm_counts/cm_counts.sum(axis=1,keepdims=True)
            print(cm_pct)
            for i in range(num_labels):
                for j in range(num_labels):
                    writer.add_scalar(f"count l{i}_p{j}",cm_counts[i,j], epoch + 1)
                    writer.add_scalar(f"pct l{i}_p{j}",cm_pct[i,j], epoch + 1)
            torch.save(model.state_dict(), f"epoch{epoch}_model{num_labels}_classification3d_array.pth")
    except KeyboardInterrupt: 
        break

print(f"Training completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
torch.save(model.state_dict(), "last_model{num_labels}_classification3d_array.pth")
writer.close()

# Model results

Here we test the model on the training/valid/test datasets, get their confusion matrices and their metrics.

In [ ]:
# create a validation data loader
test_ds = ImageDataset(image_files=test_images, labels=test_labels, transform=val_transforms)
test_loader = DataLoader(test_ds, batch_size=2, num_workers=2, pin_memory=pin_memory)

In [ ]:
len(test_images)

## Last model

In [ ]:
model.load_state_dict(torch.load(f"last_model{num_labels}_classification3d_array.pth"))

In [ ]:
v_label, v_pred = get_predicted_values(model, train_loader)
conf_matrix = get_cm(v_label, v_pred, num_labels)
all_metrics = {
    metric: metrics_fns[metric](v_label, v_pred) for metric in metrics_fns
}
conf_matrix, all_metrics

In [ ]:
v_label, v_pred = get_predicted_values(model, val_loader)
conf_matrix = get_cm(v_label, v_pred, num_labels)
all_metrics = {
    metric: metrics_fns[metric](v_label, v_pred) for metric in metrics_fns
}
conf_matrix, all_metrics

In [ ]:
v_label, v_pred = get_predicted_values(model, test_loader)
conf_matrix = get_cm(v_label, v_pred, num_labels)
all_metrics = {
    metric: metrics_fns[metric](v_label, v_pred) for metric in metrics_fns
}
conf_matrix, all_metrics

## Best model

In [ ]:
model.load_state_dict(torch.load(f"best_metric_model{num_labels}_classification3d_array.pth"))

In [ ]:
v_label, v_pred = get_predicted_values(model, train_loader)
conf_matrix = get_cm(v_label, v_pred, num_labels)
all_metrics = {
    metric: metrics_fns[metric](v_label, v_pred) for metric in metrics_fns
}
conf_matrix, all_metrics

In [ ]:
v_label, v_pred = get_predicted_values(model, val_loader)
conf_matrix = get_cm(v_label, v_pred, num_labels)
all_metrics = {
    metric: metrics_fns[metric](v_label, v_pred) for metric in metrics_fns
}
conf_matrix, all_metrics

In [ ]:
v_label, v_pred = get_predicted_values(model, test_loader)
conf_matrix = get_cm(v_label, v_pred, num_labels)
all_metrics = {
    metric: metrics_fns[metric](v_label, v_pred) for metric in metrics_fns
}
conf_matrix, all_metrics

In [ ]:
model.load_state_dict(torch.load(f"epoch{26}_model{num_labels}_classification3d_array.pth"))

In [ ]:
v_label, v_pred = get_predicted_values(model, train_loader)
conf_matrix = get_cm(v_label, v_pred, num_labels)
all_metrics = {
    metric: metrics_fns[metric](v_label, v_pred) for metric in metrics_fns
}
conf_matrix, all_metrics

In [ ]:
v_label, v_pred = get_predicted_values(model, val_loader)
conf_matrix = get_cm(v_label, v_pred, num_labels)
all_metrics = {
    metric: metrics_fns[metric](v_label, v_pred) for metric in metrics_fns
}
conf_matrix, all_metrics

In [ ]:
v_label, v_pred = get_predicted_values(model, test_loader)
conf_matrix = get_cm(v_label, v_pred, num_labels)
all_metrics = {
    metric: metrics_fns[metric](v_label, v_pred) for metric in metrics_fns
}
conf_matrix, all_metrics

# Full training

Train the model on all images for external use

In [ ]:
full_images = np.concatenate([images, test_images])
full_labels = np.concatenate([labels, test_labels])

full_images.shape, full_labels.shape

In [ ]:
val_pct = 0.1
val_split = int(val_pct*len(full_labels))
possible_labels = sorted(list(set(full_labels)))
proportion_in_labels = np.array([sum(labels==i)/len(labels) for i in possible_labels])
val_per_labels = [int(l*val_split) for l in proportion_in_labels]
proportion_in_labels, val_per_labels

In [ ]:
label_weigths = 1/proportion_in_labels[labels]

In [ ]:
val_idx = np.concatenate([np.random.choice([i for i, l in enumerate(labels) if l==p], c, replace=False) for p,c in zip(possible_labels, val_per_labels)])
in_val = np.isin(np.arange(len(labels)),val_idx)
in_train = ~in_val
train_idx = np.arange(len(labels))[in_train]
val_idx

In [ ]:
batch_size = 4#3
while (len(full_images)-val_split)%batch_size==1 or val_split%batch_size==1:
    batch_size +=1
    print("Changing batch size so that no batch has size 1")
batch_size

In [ ]:
# create a training data loader
train_ds = ImageDataset(image_files=full_images[train_idx].tolist(), labels=full_labels[train_idx].tolist(), transform=train_transforms)
train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=pin_memory)

# create a validation data loader
val_ds = ImageDataset(image_files=full_images[val_idx].tolist(), labels=full_labels[val_idx].tolist(), transform=val_transforms)
val_loader = DataLoader(val_ds, batch_size=batch_size, num_workers=2, pin_memory=pin_memory)

In [ ]:
# Create DenseNet121, CrossEntropyLoss and Adam optimizer
model = monai.networks.nets.DenseNet(spatial_dims=3, in_channels=1, out_channels=num_labels).to(device)

loss_function = torch.nn.CrossEntropyLoss(torch.tensor(1/proportion_in_labels, device=device, dtype=torch.float32))
# loss_function = torch.nn.BCEWithLogitsLoss()  # also works with this data

optimizer = torch.optim.Adam(model.parameters(), 1e-4)

# start a typical PyTorch training
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
writer = SummaryWriter(f"full_runs/{datetime.datetime.now():%Y-%m-%d_%H:%M:%S}_{socket.gethostname()}")
max_epochs = 256

for epoch in range(max_epochs):
    try:
        print("-" * 10)
        print(f"epoch {epoch + 1}/{max_epochs}")
        model.train()
        epoch_loss = 0
        step = 0

        for batch_data in train_loader:
            step += 1
            b_inputs, b_labels = batch_data[0].to(device), batch_data[1].to(device)
            optimizer.zero_grad()
            outputs = model(b_inputs)
            loss = loss_function(outputs, b_labels)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            epoch_len = len(train_ds) // train_loader.batch_size
            print(f"{step}/{epoch_len}, train_loss: {loss.item():.4f}")
            writer.add_scalar("train_loss", loss.item(), epoch_len * epoch + step)

        epoch_loss /= step
        epoch_loss_values.append(epoch_loss)
        print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

        if (epoch + 1) % val_interval == 0:
            model.eval()

            num_correct = 0.0
            metric_count = 0
            v_pred, v_label = get_predicted_values(model, val_loader)
            cm_counts = get_cm(v_pred, v_label, num_labels)

            all_metrics = {
                metric: metrics_fns[metric](v_label, v_pred) for metric in metrics_fns
            }
            metric = all_metrics[val_metric]
            metric_values.append(metric)

            if metric > best_metric:
                best_metric = metric
                best_metric_epoch = epoch + 1
                torch.save(model.state_dict(), "full_best_metric_model_classification3d_array.pth")
                print("saved new best metric model")

            print(f"Current epoch: {epoch+1} current {val_metric}: {metric:.4f} ")
            print(f"Best {val_metric}: {best_metric:.4f} at epoch {best_metric_epoch}")
            for metric in all_metrics:
                writer.add_scalar(f"val_{metric}", all_metrics[metric], epoch + 1)
            cm_pct = cm_counts/cm_counts.sum(axis=1,keepdims=True)
            print(cm_pct)
            for i in range(num_labels):
                for j in range(num_labels):
                    writer.add_scalar(f"count l{i}_p{j}",cm_counts[i,j], epoch + 1)
                    writer.add_scalar(f"pct l{i}_p{j}",cm_pct[i,j], epoch + 1)
    except KeyboardInterrupt: 
        break

print(f"Training completed, best_metric: {best_metric:.4f} at epoch: {best_metric_epoch}")
torch.save(model.state_dict(), "full_last_model_classification3d_array.pth")
writer.close()

## External Validation

In [ ]:
eval_dir = os.path.expanduser(os.path.expandvars(os.path.join("~/data/shade2022/validation_cleaned","covid/images")))
eval_img_paths = []
for data_handle in [f for f in os.listdir(eval_dir) if not f.startswith(".DS_Store")]:
    data_dir = os.path.join(eval_dir, data_handle)
    eval_img_paths.append(os.path.join(data_dir, [f for f in os.listdir(data_dir) if not f.startswith(".DS_Store")][0]))

# create a validation data loader
eval_ds = ImageDataset(image_files=np.array(eval_img_paths), labels=np.array([0]*len(eval_img_paths)), transform=val_transforms)
eval_loader = DataLoader(eval_ds, batch_size=2, num_workers=2, pin_memory=pin_memory)

v_label, v_pred = get_predicted_values(model, train_loader)
print(*list(zip(map(lambda p: p.split("/")[-2], eval_img_paths), v_pred)))

In [ ]:
eval_dir = os.path.expanduser(os.path.expandvars(os.path.join("~/data/shade2022/validation_cleaned","kidney/images")))
eval_img_paths = []
for data_handle in [f for f in os.listdir(eval_dir) if not f.startswith(".DS_Store")]:
    data_dir = os.path.join(eval_dir, data_handle)
    eval_img_paths.append(os.path.join(data_dir, [f for f in os.listdir(data_dir) if not f.startswith(".DS_Store")][0]))

# create a validation data loader
eval_ds = ImageDataset(image_files=np.array(eval_img_paths), labels=np.array([0]*len(eval_img_paths)), transform=val_transforms)
eval_loader = DataLoader(eval_ds, batch_size=2, num_workers=2, pin_memory=pin_memory)

v_label, v_pred = get_predicted_values(model, train_loader)
print(*list(zip(map(lambda p: p.split("/")[-2], eval_img_paths), v_pred)))